In [1]:
# Input info
#se_f = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/gff_A2_black/annotation_clones/SE.rds"
outdir = "/data/Mito_Trace/output/aggregate/CHIP_aggr/results/clones/variants.init_knnRes.30/aggregate/clone_cluster_sizes"
se_cells_meta_f = "/data/Mito_Trace/output/aggregate/CHIP_aggr/data/annotation/gff_A2/mergedSamples/se_cells_meta.tsv"
cells_meta_f = "/data/Mito_Trace/output/aggregate/CHIP_aggr/results/clones/variants.init_knnRes.30/aggregate/cells_meta.tsv"
min_cell = 1
keep_top = 20
keep_top_single = 10 # for the plots with same donor

input_col = "Input"
use_input = False

#sample_names = "Control,Flt3l,Input"



In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from os.path import join
%matplotlib inline

import numpy as np

In [3]:
def keep_top_clones(df, n_top=None, min_cell=0):
    df = df.sort_values("ncells",ascending=False)
    if n_top is not None:
        df = df[:n_top]
    return df.loc[df["ncells"]>min_cell]

## Load cells_meta and rename cluster labels to replace '/' with '_'

In [4]:
se_cells_meta = pd.read_csv(se_cells_meta_f, sep='\t')
#se_cells_meta = se_cells_meta.rename({"cluster_labels":"nuclear_clusters"}, axis=1)
exp_name_map = {'CHIP_a11_Input':'CHIP_a1_Input', 'CHIP_a12_Flt3l':'CHIP_a1_Flt3l', 'CHIP_a13_Input':'CHIP_a1_Input', 
                'CHIP_b11_Control':'CHIP_b1_Control','CHIP_b12_Flt3l': 'CHIP_b1_Flt3l', 'CHIP_b13_Input': 'CHIP_b1_Input',
                'cd34norm1_Control':'cd34norm_Control', 'cd34norm1_Flt3l':'cd34norm_Flt3l'}
exp_name_map

def change_name(x):
    for e in exp_name_map:
        if e in x:
            x = x.replace(e, exp_name_map[e])
            return x
    return x
se_cells_meta.index = [change_name(x) for x in se_cells_meta.index]
se_cells_meta

,orig.ident,nCount_ATAC,nFeature_ATAC,total,duplicate,chimeric,unmapped,lowmapq,mitochondrial,passed_filters,...,pct_reads_in_peaks,blacklist_ratio,high.tss,nucleosome_group,ATAC_snn_res.0.8,seurat_clusters,nCount_RNA,nFeature_RNA,ID,nuclear_clusters
cd34norm_Control_AAACGAAAGAGGTCCA-1,Control,1507,1507,16929.0,2587,58,70,674,9629,3911,...,34.006648,0,High,NS < 4,4,4,2025,1719,cd34norm1_Control_AAACGAAAGAGGTCCA-1,4
cd34norm_Control_AAACGAAAGCGATACG-1,Control,4003,4003,45359.0,10546,184,184,2302,22247,9896,...,34.983832,0,High,NS < 4,0,0,5299,3845,cd34norm1_Control_AAACGAAAGCGATACG-1,0
cd34norm_Control_AAACGAAAGTCGTGAG-1,Control,1199,1199,10177.0,2450,48,60,657,2135,4827,...,20.012430,0,High,NS < 4,1,1,2066,1694,cd34norm1_Control_AAACGAAAGTCGTGAG-1,1
cd34norm_Control_AAACGAACAATAGTGA-1,Control,2950,2950,22452.0,4364,100,83,897,9233,7775,...,34.739550,0,High,NS < 4,0,0,3746,2775,cd34norm1_Control_AAACGAACAATAGTGA-1,0
cd34norm_Control_AAACGAACACAATAAG-1,Control,1398,1398,11763.0,2003,43,57,680,5547,3433,...,33.148849,0,High,NS < 4,4,4,1866,1641,cd34norm1_Control_AAACGAACACAATAAG-1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CHIP_b2_Input_TTTGTGTGTGGAAAGA-1,Input,6856,6856,35511.0,7946,148,118,1467,14501,11331,...,58.767982,0,High,NS < 4,10,10,8326,5561,CHIP_b2_Input_TTTGTGTGTGGAAAGA-1,10
CHIP_b2_Input_TTTGTGTGTGGACAGT-1,Input,1880,1880,14217.0,2190,42,40,420,8229,3296,...,51.911408,0,High,NS < 4,1,1,1963,1704,CHIP_b2_Input_TTTGTGTGTGGACAGT-1,1
CHIP_b2_Input_TTTGTGTTCAGTACAC-1,Input,2998,2998,15165.0,2707,67,52,475,7195,4669,...,60.484044,0,High,NS < 4,1,1,3235,2634,CHIP_b2_Input_TTTGTGTTCAGTACAC-1,1
CHIP_b2_Input_TTTGTGTTCTATAGGC-1,Input,8513,8513,48687.0,14365,242,206,2591,7487,23796,...,34.644478,0,High,NS < 4,11,11,14161,7871,CHIP_b2_Input_TTTGTGTTCTATAGGC-1,11


In [5]:
cells_meta = pd.read_csv(cells_meta_f, sep='\t', index_col=0)
cells_meta

print('duplicated before adding exp name', cells_meta.index.duplicated().sum())

cells_meta['exp'] = ["_".join(x.split("_")[:-1]) for x in cells_meta["donor"].values]
cells_meta.index= cells_meta.apply(lambda x: f'{x["exp"]}_{x.name}', axis=1)
cells_meta
#cells_meta['donor'] = 

print('duplicated after adding exp name', cells_meta.index.duplicated().sum())


duplicated before adding exp name 300
duplicated after adding exp name 0


## merge cells meta and cluster labels

In [6]:
cells_merge = pd.merge(cells_meta[["name", "donor", "condition", "lineage", "exp"]], se_cells_meta, left_index=True, right_index=True, how='outer')
cells_merge

,name,donor,condition,lineage,exp,orig.ident,nCount_ATAC,nFeature_ATAC,total,duplicate,...,pct_reads_in_peaks,blacklist_ratio,high.tss,nucleosome_group,ATAC_snn_res.0.8,seurat_clusters,nCount_RNA,nFeature_RNA,ID,nuclear_clusters
CHIP_a1_Flt3l_AAACGAATCTTACTCA-1,CHIP_a1_None,CHIP_a1_None,None,None,CHIP_a1,Flt3l,220.0,220.0,779.0,47.0,...,51.174935,0.0,High,NS < 4,8.0,8.0,257.0,252.0,CHIP_a12_Flt3l_AAACGAATCTTACTCA-1,8.0
CHIP_a1_Flt3l_AAACTCGAGTGGTGTG-1,CHIP_a1_None,CHIP_a1_None,None,None,CHIP_a1,Flt3l,398.0,398.0,16911.0,5224.0,...,46.200241,0.0,High,NS < 4,1.0,1.0,515.0,490.0,CHIP_a12_Flt3l_AAACTCGAGTGGTGTG-1,1.0
CHIP_a1_Flt3l_AAACTCGTCAATGCAC-1,CHIP_a1_None,CHIP_a1_None,None,None,CHIP_a1,Flt3l,1025.0,1025.0,11963.0,2864.0,...,67.025572,0.0,High,NS < 4,8.0,8.0,1114.0,1060.0,CHIP_a12_Flt3l_AAACTCGTCAATGCAC-1,8.0
CHIP_a1_Flt3l_AAAGATGAGCGCATTT-1,CHIP_a1_None,CHIP_a1_None,None,None,CHIP_a1,Flt3l,377.0,377.0,15367.0,5530.0,...,54.142012,0.0,High,NS < 4,8.0,8.0,472.0,451.0,CHIP_a12_Flt3l_AAAGATGAGCGCATTT-1,8.0
CHIP_a1_Flt3l_AAAGATGCAGCAACGA-1,CHIP_a1_4_9,CHIP_a1_4,Flt3l,9,CHIP_a1,Flt3l,10650.0,10650.0,250347.0,85267.0,...,68.695816,0.0,High,NS < 4,5.0,5.0,17199.0,8217.0,CHIP_a12_Flt3l_AAAGATGCAGCAACGA-1,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cd34norm_Flt3l_TTTGTGTTCGAGTTAC-1,cd34norm_3_40,cd34norm_3,Flt3l,40,cd34norm,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cd34norm_Flt3l_TTTGTGTTCGCATAAC-1,cd34norm_None,cd34norm_None,None,None,cd34norm,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cd34norm_Flt3l_TTTGTGTTCGTGGTAT-1,cd34norm_1_2,cd34norm_1,Flt3l,2,cd34norm,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cd34norm_Flt3l_TTTGTGTTCGTTACAG-1,cd34norm_2_56,cd34norm_2,Flt3l,56,cd34norm,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# cells_merge =pd.merge(cells_meta[["name", "condition", "donor", "lineage", "exp"]], 
#                       se_cells_meta, left_index=True, right_index=True, how='inner')
# cells_merge

### Remove None before normalizing

In [8]:
if use_input:
    cells_merge = cells_merge[cells_merge["orig.ident"] == input_col]
cells_merge = cells_merge[cells_merge["donor"] != "None"]
cells_merge = cells_merge[cells_merge["nuclear_clusters"] != "None"]
cells_merge = cells_merge.loc[~(cells_merge["name"].fillna("").str.contains("_None"))]
cells_merge

,name,donor,condition,lineage,exp,orig.ident,nCount_ATAC,nFeature_ATAC,total,duplicate,...,pct_reads_in_peaks,blacklist_ratio,high.tss,nucleosome_group,ATAC_snn_res.0.8,seurat_clusters,nCount_RNA,nFeature_RNA,ID,nuclear_clusters
CHIP_a1_Flt3l_AAAGATGCAGCAACGA-1,CHIP_a1_4_9,CHIP_a1_4,Flt3l,9,CHIP_a1,Flt3l,10650.0,10650.0,250347.0,85267.0,...,68.695816,0.0,High,NS < 4,5.0,5.0,17199.0,8217.0,CHIP_a12_Flt3l_AAAGATGCAGCAACGA-1,5.0
CHIP_a1_Flt3l_AAAGGATGTGCTGTGC-1,CHIP_a1_0_7,CHIP_a1_0,Flt3l,7,CHIP_a1,Flt3l,9337.0,9337.0,168445.0,47237.0,...,79.218022,0.0,High,NS < 4,2.0,2.0,11654.0,6967.0,CHIP_a12_Flt3l_AAAGGATGTGCTGTGC-1,2.0
CHIP_a1_Flt3l_AAAGGATTCAAAGTAG-1,CHIP_a1_1_1,CHIP_a1_1,Flt3l,1,CHIP_a1,Flt3l,10588.0,10588.0,259299.0,60300.0,...,71.279346,0.0,High,NS < 4,1.0,1.0,14478.0,7963.0,CHIP_a12_Flt3l_AAAGGATTCAAAGTAG-1,1.0
CHIP_a1_Flt3l_AAATGCCCAGAAAGAG-1,CHIP_a1_3_8,CHIP_a1_3,Flt3l,8,CHIP_a1,Flt3l,9273.0,9273.0,165196.0,61968.0,...,78.518519,0.0,High,NS < 4,12.0,12.0,12866.0,7424.0,CHIP_a12_Flt3l_AAATGCCCAGAAAGAG-1,12.0
CHIP_a1_Flt3l_AAATGCCGTAAGGTCG-1,CHIP_a1_1_4,CHIP_a1_1,Flt3l,4,CHIP_a1,Flt3l,8753.0,8753.0,162883.0,52874.0,...,64.693095,0.0,High,NS < 4,16.0,16.0,13131.0,7006.0,CHIP_a12_Flt3l_AAATGCCGTAAGGTCG-1,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cd34norm_Flt3l_TTTGTGTTCCTTTGCG-1,cd34norm_1_0,cd34norm_1,Flt3l,0,cd34norm,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cd34norm_Flt3l_TTTGTGTTCGAGTTAC-1,cd34norm_3_40,cd34norm_3,Flt3l,40,cd34norm,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cd34norm_Flt3l_TTTGTGTTCGTGGTAT-1,cd34norm_1_2,cd34norm_1,Flt3l,2,cd34norm,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cd34norm_Flt3l_TTTGTGTTCGTTACAG-1,cd34norm_2_56,cd34norm_2,Flt3l,56,cd34norm,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Get counts of clusters for each donor

In [9]:
clone_counts = cells_merge.groupby(["name", "donor"]).size().reset_index().rename({0:"ncells"}, axis=1)
clone_counts

,name,donor,ncells
0,CHIP_a1_0_0,CHIP_a1_0,156
1,CHIP_a1_0_1,CHIP_a1_0,112
2,CHIP_a1_0_10,CHIP_a1_0,4
3,CHIP_a1_0_11,CHIP_a1_0,4
4,CHIP_a1_0_12,CHIP_a1_0,4
...,...,...,...
757,cd34norm_3_56,cd34norm_3,4
758,cd34norm_3_6,cd34norm_3,120
759,cd34norm_3_7,cd34norm_3,94
760,cd34norm_3_8,cd34norm_3,91


In [10]:
cells_merge.columns

Index(['name', 'donor', 'condition', 'lineage', 'exp', 'orig.ident',
       'nCount_ATAC', 'nFeature_ATAC', 'total', 'duplicate', 'chimeric',
       'unmapped', 'lowmapq', 'mitochondrial', 'passed_filters', 'cell_id',
       'is__cell_barcode', 'TSS_fragments', 'DNase_sensitive_region_fragments',
       'enhancer_region_fragments', 'promoter_region_fragments',
       'on_target_fragments', 'blacklist_region_fragments',
       'peak_region_fragments', 'peak_region_cutsites', 'proj', 'expID',
       'nucleosome_signal', 'nucleosome_percentile', 'TSS.enrichment',
       'TSS.percentile', 'pct_reads_in_peaks', 'blacklist_ratio', 'high.tss',
       'nucleosome_group', 'ATAC_snn_res.0.8', 'seurat_clusters', 'nCount_RNA',
       'nFeature_RNA', 'ID', 'nuclear_clusters'],
      dtype='object')

In [11]:
clone_counts_mt = cells_merge.groupby(["name", "donor"])[["nCount_ATAC"]]
clone_counts_mt = clone_counts_mt.mean().reset_index()
clone_counts_mt
#.size().reset_index().rename({0:"ncells"}, axis=1)

,name,donor,nCount_ATAC
0,CHIP_a1_0_0,CHIP_a1_0,3590.096154
1,CHIP_a1_0_1,CHIP_a1_0,4156.187500
2,CHIP_a1_0_10,CHIP_a1_0,5500.250000
3,CHIP_a1_0_11,CHIP_a1_0,2619.750000
4,CHIP_a1_0_12,CHIP_a1_0,5865.250000
...,...,...,...
757,cd34norm_3_56,cd34norm_3,NaN
758,cd34norm_3_6,cd34norm_3,1334.848485
759,cd34norm_3_7,cd34norm_3,5204.875000
760,cd34norm_3_8,cd34norm_3,4397.822222


In [12]:
group_counts_sum = clone_counts.groupby("donor").sum()["ncells"]

clone_counts["norm"] = clone_counts.apply(lambda x: x["ncells"]/group_counts_sum[x["donor"]], axis=1)
clone_counts = pd.merge(clone_counts_mt, clone_counts, on=["name", "donor"])
clone_counts

,name,donor,nCount_ATAC,ncells,norm
0,CHIP_a1_0_0,CHIP_a1_0,3590.096154,156,0.198726
1,CHIP_a1_0_1,CHIP_a1_0,4156.187500,112,0.142675
2,CHIP_a1_0_10,CHIP_a1_0,5500.250000,4,0.005096
3,CHIP_a1_0_11,CHIP_a1_0,2619.750000,4,0.005096
4,CHIP_a1_0_12,CHIP_a1_0,5865.250000,4,0.005096
...,...,...,...,...,...
757,cd34norm_3_56,cd34norm_3,NaN,4,0.000965
758,cd34norm_3_6,cd34norm_3,1334.848485,120,0.028951
759,cd34norm_3_7,cd34norm_3,5204.875000,94,0.022678
760,cd34norm_3_8,cd34norm_3,4397.822222,91,0.021954


In [13]:
summary_clone_counts = clone_counts.loc[~(clone_counts["name"].str.contains("None"))].groupby("donor").describe()
summary_clone_counts

nCount_ATAC                                                      \
                 count         mean          std          min          25%   
donor                                                                        
CHIP_a1_0         47.0  4424.007455  1284.858647  2619.750000  3657.933333   
CHIP_a1_1         49.0  4271.090093  1185.544988  1912.500000  3616.666667   
CHIP_a1_2         73.0  4382.726153  1096.435634  2438.666667  3866.500000   
CHIP_a1_3         28.0  4269.064342  1288.871774  1790.500000  3499.033071   
CHIP_a1_4         52.0  4755.255539  1476.699311  2699.400000  3828.175000   
CHIP_b1_0         34.0  3658.322435   939.341461  1525.152975  3409.477238   
CHIP_b1_1         33.0  3174.985994   738.123099  1414.534799  2863.485714   
CHIP_b2_0         78.0  2764.410414   857.935677   874.500000  2268.362745   
CHIP_b2_1         74.0  3008.401458   936.680194  1264.500000  2386.500000   
cd34norm_0       136.0  1817.470810   879.883881   336.000000  1214.645833   
cd34norm_1        34.0  2188.329904  1054.779617   417.000000  1545.875000   
cd34norm_2        46.0  2434.065371  1849.453210   420.000000  1180.375000   
cd34norm_3        47.0  2445.528439  1544.789344   379.000000  1444.666667   

                                                  ncells              ...  \
                    50%          75%          max  count        mean  ...   
donor                                                                 ...   
CHIP_a1_0   4077.000000  4776.600000  9609.666667   47.0   16.702128  ...   
CHIP_a1_1   4035.153846  4933.333333  7154.636364   49.0   22.326531  ...   
CHIP_a1_2   4114.800000  4662.333333  9806.000000   73.0   12.808219  ...   
CHIP_a1_3   4048.621212  4473.766304  7525.750000   28.0   26.892857  ...   
CHIP_a1_4   4339.916667  5105.533333  9233.428571   52.0   13.730769  ...   
CHIP_b1_0   3638.244344  4245.394995  5311.031390   34.0  254.705882  ...   
CHIP_b1_1   3140.258621  3675.087912  4647.811475   33.0  241.787879  ...   
CHIP_b2_0   2705.647826  3212.273810  5535.000000   78.0   30.064103  ...   
CHIP_b2_1   2790.138228  3507.520833  5519.500000   74.0   23.202703  ...   
cd34norm_0  1651.777778  2182.921053  5355.166667  140.0   21.450000  ...   
cd34norm_1  2032.028571  2889.821154  4262.062500   37.0  102.540541  ...   
cd34norm_2  2106.166667  2850.416667  9700.000000   60.0   61.433333  ...   
cd34norm_3  2214.762332  2941.166667  9420.000000   57.0   72.719298  ...   

                             norm                                          \
               75%     max  count      mean       std       min       25%   
donor                                                                       
CHIP_a1_0    14.50   156.0   47.0  0.021277  0.036262  0.002548  0.005096   
CHIP_a1_1    13.00   171.0   49.0  0.020408  0.036181  0.001828  0.002742   
CHIP_a1_2     7.00   141.0   73.0  0.013699  0.030048  0.001070  0.003209   
CHIP_a1_3    31.50   133.0   28.0  0.035714  0.044125  0.002656  0.006308   
CHIP_a1_4    13.25    69.0   52.0  0.019231  0.024519  0.002801  0.004202   
CHIP_b1_0   330.25   925.0   34.0  0.029412  0.019859  0.007968  0.014636   
CHIP_b1_1   309.00   648.0   33.0  0.030303  0.019627  0.007269  0.016794   
CHIP_b2_0    41.75   288.0   78.0  0.012821  0.019763  0.000853  0.002132   
CHIP_b2_1    30.50   159.0   74.0  0.013514  0.018679  0.001165  0.002330   
cd34norm_0   11.00   395.0  140.0  0.007143  0.017754  0.000333  0.000999   
cd34norm_1  139.00   557.0   37.0  0.027027  0.037415  0.000527  0.002372   
cd34norm_2   36.50  1214.0   60.0  0.016667  0.049165  0.000543  0.000814   
cd34norm_3   44.00  1409.0   57.0  0.017544  0.050143  0.000483  0.001206   

                                          
                 50%       75%       max  
donor                                     
CHIP_a1_0   0.008917  0.018471  0.198726  
CHIP_a1_1   0.003656  0.011883  0.156307  
CHIP_a1_2   0.004278  0.007487  0.150802  
CHIP_a1_3   0.017928  0.041833  0.1766

In [14]:
tmp_clone_counts = clone_counts.loc[~(clone_counts["name"].str.contains("None"))]
tmp_clone_counts= tmp_clone_counts.loc[tmp_clone_counts["ncells"]>10]
tmp_clone_counts

summary_clone_counts_above_ten = tmp_clone_counts.groupby("donor").describe()
summary_clone_counts_above_ten

nCount_ATAC                                                      \
                 count         mean          std          min          25%   
donor                                                                        
CHIP_a1_0         17.0  4339.138399  1230.816616  3380.666667  3590.096154   
CHIP_a1_1         15.0  4310.128113  1024.778354  3440.869565  3700.874415   
CHIP_a1_2         13.0  4158.874116   447.166313  3540.242424  3879.000000   
CHIP_a1_3         16.0  4568.209474  1420.511935  3246.538462  3667.374878   
CHIP_a1_4         15.0  5322.831344  2118.786759  3082.826923  3905.004963   
CHIP_b1_0         34.0  3658.322435   939.341461  1525.152975  3409.477238   
CHIP_b1_1         33.0  3174.985994   738.123099  1414.534799  2863.485714   
CHIP_b2_0         39.0  2993.595055   636.485844  1875.826923  2533.125000   
CHIP_b2_1         34.0  2809.129433   507.682112  1995.460317  2440.486743   
cd34norm_0        37.0  1820.772502   606.497003   831.800000  1360.600000   
cd34norm_1        24.0  2386.243752  1035.885571   672.000000  1700.107143   
cd34norm_2        21.0  2513.472558  1197.544858   946.411765  2047.630769   
cd34norm_3        27.0  2633.928517  1190.853611   840.285714  1955.083333   

                                                  ncells              ...  \
                    50%          75%          max  count        mean  ...   
donor                                                                 ...   
CHIP_a1_0   4054.090909  4434.027778  8593.419355   17.0   37.941176  ...   
CHIP_a1_1   3916.084337  4432.444356  7154.636364   15.0   64.466667  ...   
CHIP_a1_2   4254.229008  4414.045455  5127.166667   13.0   53.076923  ...   
CHIP_a1_3   3998.997152  4859.548913  7525.750000   16.0   43.687500  ...   
CHIP_a1_4   4515.711111  6646.791498  9233.428571   15.0   35.533333  ...   
CHIP_b1_0   3638.244344  4245.394995  5311.031390   34.0  254.705882  ...   
CHIP_b1_1   3140.258621  3675.087912  4647.811475   33.0  241.787879  ...   
CHIP_b2_0   2913.324324  3219.959261  4685.116279   39.0   55.102564  ...   
CHIP_b2_1   2714.376417  3196.012617  3867.090909   34.0   45.352941  ...   
cd34norm_0  1652.555556  2170.894737  3147.538462   37.0   68.918919  ...   
cd34norm_1  2290.714514  3124.224619  4262.062500   25.0  148.600000  ...   
cd34norm_2  2230.416667  2799.666667  5933.000000   22.0  160.045455  ...   
cd34norm_3  2335.000000  3246.333333  5235.500000   27.0  147.592593  ...   

                            norm                                          \
               75%     max count      mean       std       min       25%   
donor                                                                      
CHIP_a1_0    36.00   156.0  17.0  0.048333  0.050486  0.014013  0.017834   
CHIP_a1_1    97.00   171.0  15.0  0.058927  0.046817  0.010055  0.022395   
CHIP_a1_2    98.00   141.0  13.0  0.056767  0.054285  0.012834  0.014973   
CHIP_a1_3    53.50   133.0  16.0  0.058018  0.047527  0.015936  0.028884   
CHIP_a1_4    51.00    69.0  15.0  0.049767  0.027541  0.015406  0.025210   
CHIP_b1_0   330.25   925.0  34.0  0.029412  0.019859  0.007968  0.014636   
CHIP_b1_1   309.00   648.0  33.0  0.030303  0.019627  0.007269  0.016794   
CHIP_b2_0    52.50   288.0  39.0  0.023498  0.023588  0.004691  0.009808   
CHIP_b2_1    54.00   159.0  34.0  0.026414  0.021278  0.006989  0.011794   
cd34norm_0   65.00   395.0  37.0  0.022950  0.029440  0.003663  0.005661   
cd34norm_1  269.00   557.0  25.0  0.039167  0.040326  0.003426  0.004217   
cd34norm_2  143.75  1214.0  22.0  0.043420  0.074878  0.002984  0.009088   
cd34norm_3  133.50  1409.0  27.0  0.035607  0.069075  0.002654  0.004463   

                                          
                 50%       75%       max  
donor                                     
CHIP_a1_0   0.028025  0.045860  0.198726  
CHIP_a1_1   0.037477  0.088665  0.156307  
CHIP_a1_2   0.029947  0.104813  0.150802  
CHIP_a1_3   0.039841  0.071049  0.176627  
CHIP_a1_4  

In [15]:
cells_merge

,name,donor,condition,lineage,exp,orig.ident,nCount_ATAC,nFeature_ATAC,total,duplicate,...,pct_reads_in_peaks,blacklist_ratio,high.tss,nucleosome_group,ATAC_snn_res.0.8,seurat_clusters,nCount_RNA,nFeature_RNA,ID,nuclear_clusters
CHIP_a1_Flt3l_AAAGATGCAGCAACGA-1,CHIP_a1_4_9,CHIP_a1_4,Flt3l,9,CHIP_a1,Flt3l,10650.0,10650.0,250347.0,85267.0,...,68.695816,0.0,High,NS < 4,5.0,5.0,17199.0,8217.0,CHIP_a12_Flt3l_AAAGATGCAGCAACGA-1,5.0
CHIP_a1_Flt3l_AAAGGATGTGCTGTGC-1,CHIP_a1_0_7,CHIP_a1_0,Flt3l,7,CHIP_a1,Flt3l,9337.0,9337.0,168445.0,47237.0,...,79.218022,0.0,High,NS < 4,2.0,2.0,11654.0,6967.0,CHIP_a12_Flt3l_AAAGGATGTGCTGTGC-1,2.0
CHIP_a1_Flt3l_AAAGGATTCAAAGTAG-1,CHIP_a1_1_1,CHIP_a1_1,Flt3l,1,CHIP_a1,Flt3l,10588.0,10588.0,259299.0,60300.0,...,71.279346,0.0,High,NS < 4,1.0,1.0,14478.0,7963.0,CHIP_a12_Flt3l_AAAGGATTCAAAGTAG-1,1.0
CHIP_a1_Flt3l_AAATGCCCAGAAAGAG-1,CHIP_a1_3_8,CHIP_a1_3,Flt3l,8,CHIP_a1,Flt3l,9273.0,9273.0,165196.0,61968.0,...,78.518519,0.0,High,NS < 4,12.0,12.0,12866.0,7424.0,CHIP_a12_Flt3l_AAATGCCCAGAAAGAG-1,12.0
CHIP_a1_Flt3l_AAATGCCGTAAGGTCG-1,CHIP_a1_1_4,CHIP_a1_1,Flt3l,4,CHIP_a1,Flt3l,8753.0,8753.0,162883.0,52874.0,...,64.693095,0.0,High,NS < 4,16.0,16.0,13131.0,7006.0,CHIP_a12_Flt3l_AAATGCCGTAAGGTCG-1,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cd34norm_Flt3l_TTTGTGTTCCTTTGCG-1,cd34norm_1_0,cd34norm_1,Flt3l,0,cd34norm,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cd34norm_Flt3l_TTTGTGTTCGAGTTAC-1,cd34norm_3_40,cd34norm_3,Flt3l,40,cd34norm,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cd34norm_Flt3l_TTTGTGTTCGTGGTAT-1,cd34norm_1_2,cd34norm_1,Flt3l,2,cd34norm,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cd34norm_Flt3l_TTTGTGTTCGTTACAG-1,cd34norm_2_56,cd34norm_2,Flt3l,56,cd34norm,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
cells_merge.columns

Index(['name', 'donor', 'condition', 'lineage', 'exp', 'orig.ident',
       'nCount_ATAC', 'nFeature_ATAC', 'total', 'duplicate', 'chimeric',
       'unmapped', 'lowmapq', 'mitochondrial', 'passed_filters', 'cell_id',
       'is__cell_barcode', 'TSS_fragments', 'DNase_sensitive_region_fragments',
       'enhancer_region_fragments', 'promoter_region_fragments',
       'on_target_fragments', 'blacklist_region_fragments',
       'peak_region_fragments', 'peak_region_cutsites', 'proj', 'expID',
       'nucleosome_signal', 'nucleosome_percentile', 'TSS.enrichment',
       'TSS.percentile', 'pct_reads_in_peaks', 'blacklist_ratio', 'high.tss',
       'nucleosome_group', 'ATAC_snn_res.0.8', 'seurat_clusters', 'nCount_RNA',
       'nFeature_RNA', 'ID', 'nuclear_clusters'],
      dtype='object')

In [17]:
clone_cluster_counts = cells_merge.groupby(["name", "nuclear_clusters","donor"])[["nCount_ATAC"]]
#clone_cluster_counts.describe()
clone_cluster_counts = clone_cluster_counts.mean().reset_index()
clone_cluster_counts

clone_cluster_counts = pd.merge(cells_merge.groupby(["name", "nuclear_clusters","donor"]).size().reset_index().rename({0:"ncells"},axis=1),
        clone_cluster_counts, on=["name", "donor", "nuclear_clusters"])
clone_cluster_counts

,name,nuclear_clusters,donor,ncells,nCount_ATAC
0,CHIP_a1_0_0,0.0,CHIP_a1_0,48,3853.187500
1,CHIP_a1_0_0,1.0,CHIP_a1_0,1,3387.000000
2,CHIP_a1_0_0,2.0,CHIP_a1_0,23,2946.956522
3,CHIP_a1_0_0,3.0,CHIP_a1_0,16,3780.500000
4,CHIP_a1_0_0,4.0,CHIP_a1_0,7,4034.714286
...,...,...,...,...,...
4539,cd34norm_3_9,3.0,cd34norm_3,1,1661.000000
4540,cd34norm_3_9,6.0,cd34norm_3,2,2189.500000
4541,cd34norm_3_9,7.0,cd34norm_3,1,466.000000
4542,cd34norm_3_9,9.0,cd34norm_3,1,835.000000


## Get clone counts for each cluster

In [18]:
clone_counts_combined = clone_counts.groupby(["name", "donor"]).sum()["ncells"].reset_index().sort_values("ncells", ascending=False).set_index("name")
def get_clone_count(x):
    curr = clone_counts[clone_counts["name"]==x.name]
    return curr.set_index("nuclear_clusters")["ncells"]
clone_counts_df = pd.concat((clone_counts_combined, clone_counts_combined.apply(get_clone_count, axis=1)),axis=1).fillna(0)

clone_counts_filt_10_df = clone_counts_df.loc[clone_counts_df["ncells"]>=min_cell]
clone_counts_filt_10_df.head()

KeyError: "None of ['nuclear_clusters'] are in the columns"

In [ ]:
def plots_donor(data, **kwargs):
    print(data.columns)
    data.drop(["ncells", "donor"],axis=1).plot.bar(stacked=True, ax=plt.gca())
    return

# Figure: Clone-cluster barplot each donor is a panel. Unshared y-axis

# Normalize per clone

In [ ]:
clone_sums = clone_counts.groupby("name").sum()["ncells"]
clone_sums

In [ ]:
# clone_norm = clone_counts.copy()
# clone_norm["ncells"] = clone_norm.apply(lambda x: x["ncells"]/clone_sums.loc[(x["name"])] , axis=1)

# print("sum of normed values", clone_norm.groupby(["nuclear_clusters", "donor"]).sum()["ncells"])

# #clone_norm = se_cells_meta.groupby(["name", "nuclear_clusters","donor"]).size().reset_index().rename({0:"ncells"}, axis=1)
# clone_norm = clone_norm[~(clone_norm["name"]=="None")]
# clone_norm_combined = clone_norm.groupby(["name", "donor"]).sum()["ncells"].reset_index().sort_values("ncells", ascending=False).set_index("name")
# def get_clone_count(x):
#     curr = clone_norm[clone_norm["name"]==x.name]
#     return curr.set_index("nuclear_clusters")["ncells"]

# clone_norm_df = pd.concat((clone_norm_combined, clone_norm_combined.apply(get_clone_count, axis=1)),axis=1).fillna(0)
# clone_norm_df.head()


# Only top clones

In [ ]:
ranks = clone_norm_df.groupby("donor")['ncells'].rank(ascending=False).astype(int)

In [ ]:

clone_norm_df['rank'] = ranks
clone_counts_df['rank'] = ranks
clone_norm_df

top_clone_norm_df = clone_norm_df[clone_norm_df["rank"]<keep_top].sort_values("rank").drop("rank",axis=1)
top_clone_norm_df

top_clone_counts_df = clone_counts_df[clone_counts_df["rank"]<keep_top].sort_values("rank").drop("rank",axis=1)
top_clone_counts_df


#cell_25 = keep_top_clones(clone_counts_filt_10_df, min_cell=min_cell).index

In [ ]:
wide_top_clone_counts_df = top_clone_counts_df.drop("ncells", axis=1).reset_index().melt(id_vars=["donor","name"], var_name="cluster",
                                                             value_name="ncells")
wide_top_clone_counts_df

wide_top_clone_norm_df = top_clone_norm_df.drop("ncells", axis=1).reset_index().melt(id_vars=["donor","name"], var_name="cluster",
                                                             value_name="ncells")
wide_top_clone_norm_df



In [ ]:
hue_order = np.sort(wide_top_clone_norm_df["donor"].unique())

---

---